# 初めにお読みください。

このシートを動かすためには、事前にCSVデータをダウンロードする必要があります。

データファイルを直接curl等でダウンロードできないため、まず、ブラウザで  
[https://docs.google.com/spreadsheets/d/e/2PACX-1vSe-8lf6l_ShJHvd126J-jGti992SUbNLu-kmJfx1IRkvma_r4DHi0bwEW89opArs8ZkSY5G2-Bc1yT/pub?gid=0&single=true&output=csv](https://docs.google.com/spreadsheets/d/e/2PACX-1vSe-8lf6l_ShJHvd126J-jGti992SUbNLu-kmJfx1IRkvma_r4DHi0bwEW89opArs8ZkSY5G2-Bc1yT/pub?gid=0&single=true&output=csv)  
へアクセスしてください。

すると、CSVファイルがダウンロードできるため、ファイル名を `COVID-19-global-data.csv` として、このREADMEを置いているフォルダへ保存してください。

Google Colaboratoryで動かすには、上記でダウンロードしたファイルを、ノートブックと同じ場所にアップロードしてください。

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from typing import Dict, List
import datetime
import json
import matplotlib.pyplot as plt
import urllib.request

# 定数

## 挙動の定義

### DATE
ある日付での都道府県ごとの件数を表示する際、日付を指定。
- ファイルの中の最終日を指定する場合はNoneを設定
- 特定の日付を指定する場合は、「YYYY-MM-DD」形式の文字列を設定

In [ ]:
DATE = None
# DATE = '2020-04-05'

### COUNTRY
ある国での日別の件数を表示する際の、国を指定。  
英語表記で指定する。  
データファイルの `ADM0_NAME` の列を参照。

In [ ]:
# 1つの都道府県だけで見たい場合：要素が1つだけのリストで定義
# COUNTRY = ['Japan']
# 緊急事態宣言を発令した都道府県（愛知県も入りそうなので入れておく）
COUNTRY = ['Japan', 'United States of America', 'China',]

### CALC_COUNTRY_FIELD
集計する国の項目

In [ ]:
COUNTRY_FIELD = 'ADM0_NAME'
DATE_FIELD = 'date'  # date_epicrvの値をdatetime型にしたものを動的に設定する列名

## 定数、変数

In [ ]:
DATA_FILE = 'COVID-19-global-data.csv'

# COVID-19のデータファイルを読み取り

In [ ]:
df = pd.read_csv(DATA_FILE)

In [ ]:
# [DEBUG] CSVの列名を表示
df.columns

In [ ]:
# [DEBUG] CSVの先頭行を表示
df.head()

In [ ]:
# [DEBUG] CSVの末尾行を表示
df.tail()

# COVID-19のデータを加工

In [ ]:
for idx, row in df.iterrows():
    # 日付のフィールド（文字列）を日付型にする
    fix_date = None
    if not pd.isnull(row['date_epicrv']):
        date_epicrv = datetime.datetime.strptime(row['date_epicrv'], '%Y-%m-%dT%H:%M:%S.%fZ')
    df.at[idx, DATE_FIELD] = date_epicrv
    # 通しに「id」の別名をつける
    
    df.at[idx, 'id'] = idx

In [ ]:
# [DEBUG] 加工したデータの先頭行を表示
df.head()

In [ ]:
# [DEBUG] データの列名を表示
# for col in df.columns:
#     print(col)

# COVID-19のデータから対象データをフィルターする
※対象外のデータを除外する

In [ ]:
df_filter = df

In [ ]:
# 居住地都道府県（英語）がNaNの行を削除する（都道府県以外の国名等が入っているもの）
df_filter = df_filter[df_filter[COUNTRY_FIELD].notnull()]

In [ ]:
# ステータスがNaN（死亡でも退院でもない）の行のみ抽出する場合は、次の行を有効にする
# df_filter = df_filter[df_filter['ステータス'].isnull()]

In [ ]:
# [DEBUG] フィルターしたデータの先頭行を表示
# df_filter.head()

In [ ]:
# [DEBUG] フィルターしたデータの列名を表示
# for col in df_filter.columns:
#     print(col)

# 件数を集計する

In [ ]:
# df_count = pd.DataFrame(df.groupby([DATE_FIELD, COUNTRY_FIELD]).count()['id']).sort_values([COUNTRY_FIELD, DATE_FIELD])

In [ ]:
# df_count = df_count.reset_index()

In [ ]:
# df_count.head()

In [ ]:
# for idx, row in df_count.iterrows():
# #     print(row)
#     pass

# クエリで使用する日付を YYYY-mm-dd 形式で設定する
この日付で都道府県別の件数を取得する

In [ ]:
query_date = DATE

if query_date is None:
    query_date = datetime.datetime.strftime(df[DATE_FIELD].max(), '%Y-%m-%d')

print('クエリで使用する日付 : %s' % (query_date,))

# グラフ描画

## 国指定、日付ごと新規感染者数

In [ ]:
# query = 'residence_pref in %s' % json.dumps(PREF)
query = '%s in %s' % (COUNTRY_FIELD, json.dumps(COUNTRY),)
df_graph = df.query(query)

In [ ]:
df_graph.head()

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 10))

fig, ax = plt.subplots(figsize=(10, 5))
# ax.set_xticklabels('', rotation=90)
plt.xticks(rotation=90)

y_max = int((int(df_graph['NewCase'].max() / 50) + 1) * 50)
ax.set(ylim = (0, y_max))

sns.lineplot(
    ax=ax,
    data=df_graph,
    x=DATE_FIELD,
    y='NewCase',
    hue=COUNTRY_FIELD)

## 国指定、日付ごと累計感染者数

In [ ]:
# query = 'residence_pref in %s' % json.dumps(PREF)
query = '%s in %s' % (COUNTRY_FIELD, json.dumps(COUNTRY),)
df_graph = df.query(query)

In [ ]:
# df_graph.head()

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 10))

fig, ax = plt.subplots(figsize=(10, 5))
# ax.set_xticklabels('', rotation=90)
plt.xticks(rotation=90)

y_max = int((int(df_graph['CumCase'].max() / 1000) + 1) * 1000)
ax.set(ylim = (0, y_max))

sns.lineplot(
    ax=ax,
    data=df_graph,
    x=DATE_FIELD,
    y='CumCase',
    hue=COUNTRY_FIELD)

## 日付指定、国ごとの新規感染者数

指定された日付（日付の指定が`None`の場合は、データファイルの最終日付）における、国別の新規感染者数をグラフ化

In [ ]:
query = '%s == %s' % (DATE_FIELD, json.dumps(query_date),)
df_graph = df.query(query).sort_values(['NewCase'], ascending=False)

In [ ]:
df_graph.head()

In [ ]:
# # [DEBUG] ファイルの最終日付における、国別の新規感染者数を一覧表示
# for idx, row in df_graph.iterrows():
#     fix_date = datetime.datetime.strftime(row[DATE_FIELD], '%Y-%m-%d')
#     print('%s,%s,%d' % (fix_date,row[COUNTRY_FIELD],row['NewCase']))

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 10))

fig, ax = plt.subplots(figsize=(50, 20))
# ax.set_xticklabels('', rotation=90)
plt.xticks(rotation=90)

y_max = (int(df_graph['NewCase'].max() / 10000) + 1) * 10000
ax.set(ylim = (0, y_max))

sns.barplot(
    ax=ax,
    data=df_graph,
    x=COUNTRY_FIELD,
    y='NewCase')

In [ ]:
print('query_date : %s' % (query_date,))

## 日付指定、国ごとの新規感染者数

指定された日付（日付の指定が`None`の場合は、データファイルの最終日付）における、国別の累積感染者数をグラフ化

In [ ]:
query = '%s == %s' % (DATE_FIELD, json.dumps(query_date),)
df_graph = df.query(query).sort_values(['CumCase'], ascending=False)

In [ ]:
df_graph.head()

In [ ]:
# # [DEBUG] ファイルの最終日付における、国別の累積感染者数を一覧表示
# for idx, row in df_graph.iterrows():
#     fix_date = datetime.datetime.strftime(row[DATE_FIELD], '%Y-%m-%d')
#     print('%s,%s,%d' % (fix_date,row[COUNTRY_FIELD],row['CumCase']))

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 10))

fig, ax = plt.subplots(figsize=(50, 20))
# ax.set_xticklabels('', rotation=90)
plt.xticks(rotation=90)

y_max = (int(df_graph['CumCase'].max() / 100000) + 1) * 100000
ax.set(ylim = (0, y_max))

sns.barplot(
    ax=ax,
    data=df_graph,
    x=COUNTRY_FIELD,
    y='CumCase')